<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction-to-pandas" data-toc-modified-id="Introduction-to-pandas-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction to pandas</a></span></li><li><span><a href="#Add-new-columns-saving-Nan-values-get_dummies_nan" data-toc-modified-id="Add-new-columns-saving-Nan-values-get_dummies_nan-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Add new columns saving Nan values <code>get_dummies_nan</code></a></span></li><li><span><a href="#Compute-nan-features-for-rows:-nan_features" data-toc-modified-id="Compute-nan-features-for-rows:-nan_features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Compute nan features for rows: <code>nan_features</code></a></span></li><li><span><a href="#Compute-nan-features-for-rows-with-respect-to-the-target-classes" data-toc-modified-id="Compute-nan-features-for-rows-with-respect-to-the-target-classes-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Compute nan features for rows with respect to the target classes</a></span></li><li><span><a href="#Make-new-columns-from-a-series-containing-lists" data-toc-modified-id="Make-new-columns-from-a-series-containing-lists-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Make new columns from a series containing lists</a></span></li></ul></div>

# Introduction to pandas

In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats

from collections.abc import Iterable 

# Add new columns saving Nan values `get_dummies_nan` 

In [2]:
df_cars = pd.DataFrame([[2,"mercedes","middleclass"], 
                        [np.NaN,"mercedes","middleclass"],
                        [3,"Audi",np.NaN]],
                        columns= ["members","vehicles","status"])


In [3]:
print(df_cars)

   members  vehicles       status
0      2.0  mercedes  middleclass
1      NaN  mercedes  middleclass
2      3.0      Audi          NaN


In [4]:

def contains_nan(df_col):
    '''
    `contains_nan` checks if a certain column has nans
    '''
    return df_col.isna().any()


def get_dummies_nan(df, return_nancols=False,inplace=False):
    '''
    `get_dummies_nan` creates a new dataframe with binary columns stating wheather variables contain NaNs.
    
    
    
    Examples:
    --------
    
    
    >>> df = pd.DataFrame([[2,"mercedes","middleclass"], 
                           [np.NaN,"mercedes","middleclass"],
                           [3,"Audi",np.NaN]],
                           columns= ["members","vehicles","status"])
    
    >>> df 

           members  vehicles       status
    0      2.0  mercedes  middleclass
    1      NaN  mercedes  middleclass
    2      3.0      Audi          NaN
    
    >>> df_ = get_dummies_nan(df)
    
    >>> df_
    
           members  vehicles       status  members_nan  status_nan
    0      2.0  mercedes  middleclass        False       False
    1      NaN  mercedes  middleclass         True       False
    2      3.0      Audi          NaN        False        True

    >>> df_, nancols = get_dummies_nan(df, return_nancols=True)
    
    >>> nancols
    
    ['members', 'status']

    '''
    if inplace:
        cols_with_nan = []
        for c in df.columns:
            if contains_nan(df[c]):
                cols_with_nan.append(c)
                df[c + "_nan"] = df[c].isna().values    
        
        if return_nancols:
            return cols_with_nan
    else:
        df_copy = df.copy(deep=True)
        cols_with_nan = []
        for c in df.columns:
            if contains_nan(df[c]):
                cols_with_nan.append(c)
                df_copy[c + "_nan"] = df[c].isna().values
        
        if return_nancols:
            return df_copy, cols_with_nan
        else:
            return df_copy

In [5]:
df_= get_dummies_nan(df_cars)

In [6]:
df_

,members,vehicles,status,members_nan,status_nan
0,2.0,mercedes,middleclass,False,False
1,NaN,mercedes,middleclass,True,False
2,3.0,Audi,NaN,False,True


In [7]:
df_, nancols = get_dummies_nan(df_cars, return_nancols=True)

In [8]:
nancols

['members', 'status']

# Compute nan features for rows: `nan_features` 

In [18]:

def _nan_rowfeatures(df, reduce_methods=[np.mean, np.std], distances=False, inplace=False):
    
    """
    `nan_features` generates new features containing for a given row k the "statistic" returned by a reduce
    operation on row k.
    
   t also computes the difference between the most value of the transformed nans per row and the found value.

    Examples:
    ---------
    >>> df = pd.DataFrame([[2,["p","b",None]], 
                   [3,["a","c",None]],
                  [3,["d","w","a"]]],columns= ["first","second"])
                  
    >>> df
    
       first        second
    0      2  [p, b, None]
    1      3  [a, c, None]
    2      3     [d, w, a]
    
    
    >>> df_ = nan_rowfeatures(df)
    
    >>> df_

        members  vehicles       status  mean_nans  std_nans
    0      2.0  mercedes  middleclass   0.000000  0.000000
    1      NaN  mercedes  middleclass   0.333333  0.471405
    2      3.0      Audi          NaN   0.333333  0.471405

    """
    def create_col(df, reducer):
        colname     = reducer.__name__ + '_rownans'
        df[colname] = df_cars.apply(lambda x: reducer(pd.isna(x)), axis=1)
        if distances:
            mode        = stats.mode(df[colname])[0]  
            df[colname + '_l1_to_mode' ] = np.abs(df[colname] - mode)
            df[colname + '_l2_to_mode']  = (df[colname] - mode)**2
            
    if inplace:
        for reducer in reduce_methods:
            create_col(df, reducer)
    else:
        df_copy = df.copy(deep=True)
        
        for reducer in reduce_methods:
            create_col(df_copy, reducer)
        return df_copy
    
    

def nan_rowfeatures(df, reduce_methods=[np.mean, np.std], distances=False, inplace=False):
    
    """
    `nan_features` generates new features containing for a given row k the "statistic" returned by a reduce
    operation on row k.
    
    It also computes the difference between the most value of the transformed nans per row and the found value.
    
    
    Examples:
    ---------
    >>> df = pd.DataFrame([[2,["p","b",None]], 
                   [3,["a","c",None]],
                  [3,["d","w","a"]]],columns= ["first","second"])
                  
    >>> df
    
       first        second
    0      2  [p, b, None]
    1      3  [a, c, None]
    2      3     [d, w, a]
    
    
    >>> df_ = nan_rowfeatures(df)
    
    >>> df_

       members  vehicles       status  mean_rownans  std_rownans
    0      2.0  mercedes  middleclass      0.000000     0.000000
    1      NaN  mercedes  middleclass      0.333333     0.433013
    2      3.0      Audi          NaN      0.333333     0.433013
    
    >>> df_ = nan_rowfeatures(df, distances=True)

           members  vehicles       status  mean_rownans  mean_rownans_l1_to_mode  \
    0      2.0  mercedes  middleclass      0.000000                 0.333333   
    1      NaN  mercedes  middleclass      0.333333                 0.000000   
    2      3.0      Audi          NaN      0.333333                 0.000000   

       mean_rownans_l2_to_mode  std_rownans  std_rownans_l1_to_mode  \
    0                 0.111111     0.000000                0.372678   
    1                 0.000000     0.372678                0.000000   
    2                 0.000000     0.372678                0.000000   

       std_rownans_l2_to_mode  
    0                0.138889  
    1                0.000000  
    2                0.000000  

    """
    
    def create_col(df, reducer):
        colname                       = reducer.__name__ + '_rownans'
        df[colname]                   = reducer(pd.isna(df),axis=1)
        if distances:
            mode                          = stats.mode(df[colname])[0]  
            df[colname + '_l1_to_mode']   = np.abs(mode - df[colname])
            df[colname + '_l2_to_mode']   = (mode - df[colname])**2
            
    if inplace:
        for reducer in reduce_methods:
            create_col(df, reducer)
    else:
        df_copy = df.copy(deep=True)
        
        for reducer in reduce_methods:
            create_col(df_copy, reducer)

        return df_copy

In [19]:
%%time
_nan_rowfeatures(df_cars)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 5.56 ms


,members,vehicles,status,mean_rownans,std_rownans
0,2.0,mercedes,middleclass,0.000000,0.000000
1,NaN,mercedes,middleclass,0.333333,0.471405
2,3.0,Audi,NaN,0.333333,0.471405


In [27]:
%%time
nan_rowfeatures(df_cars)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 8.8 ms


,members,vehicles,status,mean_rownans,std_rownans
0,2.0,mercedes,middleclass,0.000000,0.000000
1,NaN,mercedes,middleclass,0.333333,0.433013
2,3.0,Audi,NaN,0.333333,0.433013


In [28]:
print(nan_rowfeatures(df_cars))

   members  vehicles       status  mean_rownans  std_rownans
0      2.0  mercedes  middleclass      0.000000     0.000000
1      NaN  mercedes  middleclass      0.333333     0.433013
2      3.0      Audi          NaN      0.333333     0.433013


In [22]:
print(nan_rowfeatures(df_cars))

   members  vehicles       status  mean_rownans  std_rownans
0      2.0  mercedes  middleclass      0.000000     0.000000
1      NaN  mercedes  middleclass      0.333333     0.433013
2      3.0      Audi          NaN      0.333333     0.433013


# Compute nan features for rows with respect to the target classes

# Make new columns from a series containing lists

Column `vehicles` contains lists with different vehicle names. Let us assume we consider this feature to be a list or the ordered vehicles a family has.

For example: Family 0 has 2 vehicles, and the most used one is a Mercedes, then a Toyota.

Now we want to create 3 features from this column: `vehicle_1`, `vehicle_2`, `vehicle_3` and write the different
models in the corresponding columns

In [ ]:
df_cars = pd.DataFrame([[2,["mercedes","toyota",None],"middleclass"], 
                        [3,["Renault","Mercedes",None],"middleclass"],
                        [3,["Audi","Mercedes","Tesla"],"uppermiddleclass"]],
                        columns= ["members","vehicles","status"])

df_cars

In [ ]:
def proc_df_collist(df: pd.DataFrame, colname: str, inplace=False):
    """
    
    `proc_df_collist` takes a dataframe and a column made of lists and generates new columns containing
    values from the lists. For each position in the list it generates a new column. The nimber of generated
    columns equals the length of the largest list in `df[colname]`. Each new collumn  k is filled
    with the values of the lists at position k. If the value does not exist (because the position does not exist)
    the position is filled with `NaN`. 
    
    Given `df`  and `colname`, create as many new columns as `len(df[colname].iloc[0])`
    Write in column `colname_k[j]` the value found `df[colname].iloc[j][k]`.
    
    
    Examples:
    ---------
    >>> df = pd.DataFrame([[2,["p","b",None]], 
                   [3,["a","c",None]],
                  [3,["d","w","a"]]],columns= ["first","second"])
                  
    >>> df
    
       first        second
    0      2  [p, b, None]
    1      3  [a, c, None]
    2      3     [d, w, a]

    >>> newcols = proc_df_collist(df, "second")

    >>> newcols
          second_0 second_1 second_2
    0        p        b     None
    1        a        c     None
    2        d        w        a

    
    >>> df2 = pd.DataFrame([[2,["p"]], 
                   [3,["a",2,3]],
                   [3,[4]]],columns= ["A","B"])
                   
    >>> df2
       A          B
    0  2        [p]
    1  3  [a, 2, 3]
    2  3        [4]

    >>> proc_df_collist(df2, "B")

      B_0  B_1  B_2
    0   p  NaN  NaN
    1   a  2.0  3.0
    2   4  NaN  NaN

    """
    assert isinstance(df, pd.DataFrame), "type(df)={} but it should be pd.DataFrame".format(type(df))
    assert isinstance(colname, str), "type(columname)={} but it should be str".format(type(str))
    assert isinstance(df[colname].iloc[0],(list,set, np.ndarray)), "type(df[colname].iloc[0])={} but it, should be in [list, set, np.ndarray]".format(type(df[colname].iloc[0]))
    
    
    #n_new_cols = len(df[colname].iloc[0])
    n_new_cols = df[colname].apply(len).max()
    colnames   = [colname + "_" + str(i) for i in range(n_new_cols)]   
    
    return pd.DataFrame(df[colname].tolist(), columns=colnames)

In [ ]:
df = pd.DataFrame([[2,["p","b",None]], 
                   [3,["a","c",None]],
                   [3,["d","w","a"]]],columns= ["first","second"])


In [ ]:
df

In [ ]:
proc_df_collist(df, "second")

If the column passed to `proc_df_collist` contains iterables with different sizes then it will generate as many colums as the longest iterable. Then it will fill with `NaN`  positions where we don't have information

In [ ]:
df2 = pd.DataFrame([[2,["p"]], 
                   [3,["a",2,3]],
                   [3,[4]]],columns= ["A","B"])

In [ ]:
proc_df_collist(df2, "B")